In [5]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.3 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import transformers
import torch

# Original Claire-7B

In [11]:

model_name = "OpenLLM-France/Claire-7B-0.1"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model_base = transformers.AutoModelForCausalLM.from_pretrained(model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    load_in_4bit=True                          # For efficient inference, if supported by the GPU card
)

pipeline_base = transformers.pipeline("text-generation", model=model_base, tokenizer=tokenizer)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [12]:

prompt = """Ci-dessous se trouve une instruction qui décrit une tâche. Écrivez une réponse qui complète de manière appropriée la demande.

### Instruction :
Détermine le sentiment associé au texte parmis positif, négatif, neutre

### Entrée
J'apprécie les fruits au sirop

### Réponse :"""

generation_kwargs = dict(
    num_return_sequences=1,                    # Number of variants to generate.
    return_full_text= False,                   # Do not include the prompt in the generated text.
    max_new_tokens=300,                        # Maximum length for the output text.
    do_sample=True, top_k=10, temperature=1.0, # Sampling parameters.
    pad_token_id=tokenizer.eos_token_id,       # Just to avoid a harmless warning.
)

completions = pipeline_base(prompt, **generation_kwargs)
for completion in completions:
    print(prompt + " […]" + completion['generated_text'])



Ci-dessous se trouve une instruction qui décrit une tâche. Écrivez une réponse qui complète de manière appropriée la demande.

### Instruction :
Détermine le sentiment associé au texte parmis positif, négatif, neutre

### Entrée
J'apprécie les fruits au sirop

### Réponse : […] -positive -neutre -negative



# Claire-7b-Instruct

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained("OpenLLM-France/Claire-7B-0.1")

In [2]:
model_name = "PhilSad/Claire-7b-0.1-instruct"

tokenizer = transformers.AutoTokenizer.from_pretrained("OpenLLM-France/Claire-7B-0.1")
model_instruct = transformers.AutoModelForCausalLM.from_pretrained(model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
pipeline_instruct = transformers.pipeline("text-generation", model=model_instruct, tokenizer=tokenizer)


In [10]:

prompt = """Ci-dessous se trouve une instruction qui décrit une tâche. Écrivez une réponse qui complète de manière appropriée la demande.

### Instruction :
Détermine le sentiment associé au texte parmis positif, négatif, neutre

### Entrée
J'apprécie les fruits au sirop

### Réponse :"""
generation_kwargs = dict(
    num_return_sequences=1,                    # Number of variants to generate.
    return_full_text= False,                   # Do not include the prompt in the generated text.
    max_new_tokens=50,                        # Maximum length for the output text.
    do_sample=True, top_k=10, temperature=0.2, # Sampling parameters.
    pad_token_id=tokenizer.eos_token_id,       # Just to avoid a harmless warning.
)
completions = pipeline_instruct(prompt, **generation_kwargs)
for completion in completions:
    print(prompt + " […]" + completion['generated_text'])



/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Ci-dessous se trouve une instruction qui décrit une tâche. Écrivez une réponse qui complète de manière appropriée la demande.

### Instruction :
Détermine le sentiment associé au texte parmis positif, négatif, neutre

### Entrée
J'apprécie les fruits au sirop

### Réponse : […]
Positif
